In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

API_KEY = 'YOUR_ALPHAVANTAGE_API_KEY'
BASE_URL = 'https://www.alphavantage.co/support/#api-key'

def fetch_stock_data(symbol, interval='1min'):
    params = {
        'function': 'TIME_SERIES_INTRADAY',
        'symbol': symbol,
        'interval': interval,
        'apikey': API_KEY
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    if 'Time Series (1min)' in data:
        return pd.DataFrame(data['Time Series (1min)']).T
    else:
        print(f"Error fetching data for {symbol}: {data.get('Error Message', 'Unknown error')}")
        return pd.DataFrame()

def add_stock_to_portfolio(portfolio, symbol, quantity):
    stock_data = fetch_stock_data(symbol)
    if not stock_data.empty:
        portfolio[symbol] = {'quantity': quantity, 'data': stock_data}
    return portfolio

def remove_stock_from_portfolio(portfolio, symbol):
    if symbol in portfolio:
        del portfolio[symbol]
    return portfolio

def track_portfolio(portfolio):
    performance = {}
    for symbol, details in portfolio.items():
        data = details['data']
        if not data.empty:
            data['4. close'] = pd.to_numeric(data['4. close'])
            current_price = data['4. close'].iloc[-1]
            performance[symbol] = current_price * details['quantity']
    return performance

def plot_portfolio_performance(portfolio):
    performance = track_portfolio(portfolio)
    symbols = list(performance.keys())
    values = list(performance.values())

    plt.figure(figsize=(10, 6))
    plt.bar(symbols, values, color='skyblue')
    plt.xlabel('Stock Symbol')
    plt.ylabel('Total Value')
    plt.title('Stock Portfolio Performance')
    plt.show()

if __name__ == "__main__":
    portfolio = {}

    portfolio = add_stock_to_portfolio(portfolio, 'AAPL', 10)
    portfolio = add_stock_to_portfolio(portfolio, 'GOOGL', 5)
    
    print("Portfolio performance:", track_portfolio(portfolio))
    plot_portfolio_performance(portfolio)
    
    portfolio = remove_stock_from_portfolio(portfolio, 'AAPL')
    print("Updated Portfolio performance:", track_portfolio(portfolio))
    plot_portfolio_performance(portfolio)
